In [1]:
import pandas as pd

In [2]:
# Load data
df = pd.read_csv(r"C:\Users\Tushar Khapre\Desktop\Data science assigment\DS Assignment batch April 2025\anime.csv")

In [3]:
df.isnull().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

In [4]:
# Drop or fill missing values (e.g., fill missing genre with 'Unknown')
df['genre'].fillna('Unknown', inplace=True)

C:\Users\Tushar Khapre\AppData\Local\Temp\ipykernel_5732\3073192814.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['genre'].fillna('Unknown', inplace=True)


In [5]:
df.shape

(12294, 7)

In [6]:
df.dropna(subset=['rating'], inplace=True)

In [7]:
df.isnull().sum()

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64

In [8]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
genre_encoded = mlb.fit_transform(df['genre'])
genre_df = pd.DataFrame(genre_encoded, columns=mlb.classes_)

In [9]:
# Combine features
features = pd.concat([genre_df, df[['rating', 'members']].reset_index(drop=True)], axis=1)

In [10]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)

In [11]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute cosine similarity
cos_sim = cosine_similarity(features_scaled)

# Function to recommend similar anime
def recommend_anime(anime_title, df, sim_matrix, top_n=5):
    idx = df[df['name'] == anime_title].index[0]
    sim_scores = list(enumerate(sim_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]  # Skip self-match
    anime_indices = [i[0] for i in sim_scores]
    return df['name'].iloc[anime_indices]

# Example
print(recommend_anime("Naruto", df, cos_sim))

615                                    Naruto: Shippuuden
1472          Naruto: Shippuuden Movie 4 - The Lost Tower
1573    Naruto: Shippuuden Movie 3 - Hi no Ishi wo Tsu...
486                              Boruto: Naruto the Movie
1343                                          Naruto x UT
Name: name, dtype: object


In [16]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

user_ratings = pd.read_csv(r"C:\Users\Tushar Khapre\Desktop\Data science assigment\DS Assignment batch April 2025\rating.csv")

# Sample 2000 users and 1000 anime for testing (adjust as needed)
sample_users = user_ratings['user_id'].drop_duplicates().sample(n=2000, random_state=42)
sample_anime = user_ratings['anime_id'].drop_duplicates().sample(n=1000, random_state=42)
user_ratings_sample = user_ratings[user_ratings['user_id'].isin(sample_users) & user_ratings['anime_id'].isin(sample_anime)]

user_anime_matrix = user_ratings_sample.pivot_table(index='user_id', columns='anime_id', values='rating')

def recommend_user_based(user_id, user_anime_matrix, top_n=5):
    user_vector = user_anime_matrix.loc[[user_id]].fillna(0)
    all_vectors = user_anime_matrix.fillna(0)
    similarities = cosine_similarity(user_vector, all_vectors)[0]
    sim_series = pd.Series(similarities, index=all_vectors.index)
    sim_series = sim_series.drop(user_id).sort_values(ascending=False)
    similar_users = sim_series.head(top_n).index
    user_watched = set(user_anime_matrix.loc[user_id].dropna().index)
    recommendations = []
    for sim_user in similar_users:
        sim_user_watched = set(user_anime_matrix.loc[sim_user].dropna().index)
        recommendations.extend(list(sim_user_watched - user_watched))
    return list(pd.Series(recommendations).value_counts().head(top_n).index)

def recommend_item_based(anime_id, user_anime_matrix, top_n=5):
    item_vector = user_anime_matrix[anime_id].fillna(0).values.reshape(1, -1)
    all_vectors = user_anime_matrix.fillna(0).T.values
    similarities = cosine_similarity(item_vector, all_vectors)[0]
    sim_series = pd.Series(similarities, index=user_anime_matrix.columns)
    sim_series = sim_series.drop(anime_id).sort_values(ascending=False)
    similar_items = sim_series.head(top_n).index
    return list(similar_items)


print(recommend_user_based(user_id=sample_users.iloc[0], user_anime_matrix=user_anime_matrix, top_n=5))
print(recommend_item_based(anime_id=sample_anime.iloc[0], user_anime_matrix=user_anime_matrix, top_n=5))

[21185, 6922, 492, 17265, 49]
[1237, 31593, 22381, 30826, 2121]
